In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-l0iiz14s
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-l0iiz14s
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-wyjm72w1/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [4]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp3_lya20b".


In [5]:
!nvidia-smi

Thu Oct 17 19:45:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [44]:
# Save the code into a .cu file
%%writefile password_crack.cu
#include <stdio.h>
#include <cuda.h>

// Custom strcmp implementation for device code
__device__ int device_strcmp(const char* str1, const char* str2) {
    int i = 0;
    while (str1[i] != '\0' && str2[i] != '\0') {
        if (str1[i] != str2[i]) {
            return 1; // Strings are different
        }
        i++;
    }
    // If both strings end at the same time, they are equal
    if (str1[i] == '\0' && str2[i] == '\0') {
        return 0;
    }
    return 1; // Strings are different in length
}

__device__ void CudaCrypt(const char* rawPassword, char* newPassword) {
    newPassword[0] = rawPassword[0] + 2;
    newPassword[1] = rawPassword[0] - 2;
    newPassword[2] = rawPassword[0] + 1;
    newPassword[3] = rawPassword[1] + 3;
    newPassword[4] = rawPassword[1] - 3;
    newPassword[5] = rawPassword[1] - 1;
    newPassword[6] = rawPassword[2] + 2;
    newPassword[7] = rawPassword[2] - 2;
    newPassword[8] = rawPassword[3] + 4;
    newPassword[9] = rawPassword[3] - 4;
    newPassword[10] = '\0';

    for (int i = 0; i < 10; i++) {
        if (i < 6) {  // Handle letters
            if (newPassword[i] > 122) {
                newPassword[i] = (newPassword[i] - 122) + 97;
            } else if (newPassword[i] < 97) {
                newPassword[i] = (97 - newPassword[i]) + 97;
            }
        } else {  // Handle digits
            if (newPassword[i] > 57) {
                newPassword[i] = (newPassword[i] - 57) + 48;
            } else if (newPassword[i] < 48) {
                newPassword[i] = (48 - newPassword[i]) + 48;
            }
        }
    }
}

// Kernel to encrypt the original password
__global__ void encryptPassword(const char* originalPassword, char* targetPassword) {
    CudaCrypt(originalPassword, targetPassword);
}

__global__ void crack(const char* targetPassword, char* result) {
    char possiblePassword[5];  // 2 letters, 2 digits + '\0'
    possiblePassword[4] = '\0';  // Null terminator

    // Generate password based on thread/block indices
    possiblePassword[0] = 'a' + blockIdx.x;  // First letter
    possiblePassword[1] = 'a' + blockIdx.y;  // Second letter
    possiblePassword[2] = '0' + threadIdx.x; // First digit
    possiblePassword[3] = '0' + threadIdx.y; // Second digit

    // Encrypt the possible password using the device version of CudaCrypt
    char encryptedPassword[11];
    CudaCrypt(possiblePassword, encryptedPassword); // Encrypt password

    // Compare with the target encrypted password
    if (device_strcmp(encryptedPassword, targetPassword) == 0) {
        // If match found, copy password to result
        for (int i = 0; i < 4; i++) {
            result[i] = possiblePassword[i];
        }
        result[4] = '\0';  // Null terminate the result
    }
}

int main(int argc, char **argv) {
    // Set the original password (to be encrypted)
    const char* originalPassword = "hp93"; // The original password to crack
    char targetPassword[11]; // Buffer for the target encrypted password

    // Allocate memory on GPU for the original password
    char* d_originalPassword;
    cudaMalloc((void**)&d_originalPassword, sizeof(char) * 5);  // 4 chars + '\0'
    cudaMemcpy(d_originalPassword, originalPassword, sizeof(char) * 5, cudaMemcpyHostToDevice);

    // Allocate memory on GPU for the target password
    char* d_targetPassword;
    cudaMalloc((void**)&d_targetPassword, sizeof(targetPassword));

    // Encrypt the original password on the GPU
    encryptPassword<<<1, 1>>>(d_originalPassword, d_targetPassword);
    cudaDeviceSynchronize();

    // Allocate memory for result on the GPU
    char* d_result;
    cudaMalloc((void**)&d_result, sizeof(char) * 5);  // 4 chars + '\0'

    // Launch kernel with a 2D grid of 26x26 blocks for letters and 10x10 threads for digits
    dim3 numBlocks(26, 26);  // 26 letters in each axis
    dim3 numThreads(10, 10); // 10 digits in each axis

    crack<<<numBlocks, numThreads>>>(d_targetPassword, d_result);

    // Wait for kernel to finish
    cudaDeviceSynchronize();

    // Copy result back to host
    char result[5];
    cudaMemcpy(result, d_result, sizeof(char) * 5, cudaMemcpyDeviceToHost);

    // Print the decrypted password
    if (result[0] != '\0') {
        printf("Password found: %s\n", result);
    } else {
        printf("Password not found.\n");
    }

    // Free memory on GPU
    cudaFree(d_originalPassword);
    cudaFree(d_targetPassword);
    cudaFree(d_result);

    return 0;
}


Overwriting password_crack.cu


In [45]:
!nvcc password_crack.cu -o password_crack
!./password_crack


Password found: hp93
